In [7]:
#Importing the required packages and creating a reddit instance so that we can use praw to obtain time resolved data.
import praw
import pandas as pd
import datetime as dt
import numpy as np
import time 
import matplotlib.pyplot as plt
%matplotlib inline

reddit = praw.Reddit(client_id='THphXqxKnVtPjg', 
                     client_secret='mSD1axW8-6e-Stm5p9eJAG5qY7Q', 
                     user_agent='RGRData', 
                     username='richgetricherdata', 
                     password='ByxnbjVnq5UykPu')

In [8]:
def score_c(idlist,scoring,timing):
    index = list(range(len(idlist)))
    for i in index:
        comment = reddit.comment(id=idlist[i])
        try:
            scoring[i,0] = comment.score
        except:
            scoring[i,0] = np.nan
        timing[i,0] = time.time()
    return scoring, timing

In [9]:
def score_s(idlist,scoring,timing):
    index = list(range(len(idlist)))
    for i in index:
        submission = reddit.submission(id=idlist[i])
        try:
            scoring[i,0] = submission.score
        except:
            scoring[i,0] = np.nan
        timing[i,0] = time.time()
    return scoring, timing

In [10]:
def comment_data(sub,n_coms,n_hours): 

    idlist = []
    scoring = np.zeros([n_coms,1])
    timing = np.zeros([n_coms,1])
    data = np.zeros([n_coms,n_hours + 1])
    times = np.zeros([n_coms,n_hours + 1])
    scoring[:,:] = np.nan
    timing[:,:] = np.nan
    data[:,:] = np.nan
    times[:,:] = np.nan
    subreddit = reddit.subreddit(sub)

    for comment in subreddit.stream.comments(skip_existing='true'):
        if len(idlist) >= n_coms:
            break
        idlist = np.append(idlist, comment.id)

    print(n_coms, "comments found at",dt.datetime.utcfromtimestamp(time.time()+3600*10.5).strftime('%Y-%m-%d %H:%M:%S'),"attempting to track for",n_hours,"hours.")  
    np.savetxt(sub + '_comment_ids_' + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.csv', idlist, delimiter=",", fmt='%s', header='IDs')

    start_time = round(time.time())
    timer = np.arange(start_time, start_time + n_hours*3600 , 3600)

    for i in range(n_hours):
        while time.time() < timer[i]:
            time.sleep(60)
        returned = score_c(idlist, scoring, timing)
        data[:,i+1] = returned[0].reshape(1,n_coms)
        times[:,i+1] = returned[1].reshape(1,n_coms)
        np.savetxt(sub + '_comment_data_' + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.csv', data, delimiter=",", fmt='%s', header='Score')
        np.savetxt(sub + '_comment_times_' + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.csv', times, delimiter=",", fmt='%s', header='Time')
        print("Scoring cycle completed at",dt.datetime.utcfromtimestamp(time.time()+3600*10.5).strftime('%Y-%m-%d %H:%M:%S'),".")

    for i in range(n_coms):
        try: 
            posted = reddit.comment(id=idlist[i])
            times[i,0] = posted.created_utc
            data[i,0] = 1
        except: 
            times[i,0] = times[i,1] - 3600

    for i in range(n_coms-1,-1,-1):
        if np.sum(np.isnan(data[i,:])) != 0:
            data = np.delete(data,i,0)
            times = np.delete(times,i,0)      

    np.savetxt(sub + '_comment_data_clean_' + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.csv', data, delimiter=",", fmt='%s', header='Score')
    np.savetxt(sub + '_comment_times_clean_' + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.csv', times, delimiter=",", fmt='%s', header='Time')

In [11]:
def post_data(sub,n_posts,n_hours): 

    idlist = []
    scoring = np.zeros([n_posts,1])
    timing = np.zeros([n_posts,1])
    data = np.zeros([n_posts,n_hours + 1])
    times = np.zeros([n_posts,n_hours + 1])
    scoring[:,:] = np.nan
    timing[:,:] = np.nan
    data[:,:] = np.nan
    times[:,:] = np.nan
    subreddit = reddit.subreddit(sub)

    for submission in subreddit.stream.submissions(skip_existing='true'):
        if len(idlist) >= n_posts:
            break
        idlist = np.append(idlist, submission.id)

    print(n_posts, "posts found at",dt.datetime.utcfromtimestamp(time.time()+3600*10.5).strftime('%Y-%m-%d %H:%M:%S'),"attempting to track for",n_hours,"hours.")  
    np.savetxt(sub + '_post_ids_' + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.csv', idlist, delimiter=",", fmt='%s', header='IDs')

    start_time = round(time.time())
    timer = np.arange(start_time, start_time + n_hours*3600 , 3600)

    for i in range(n_hours):
        while time.time() < timer[i]:
            time.sleep(60)
        returned = score_s(idlist, scoring, timing)
        data[:,i+1] = returned[0].reshape(1,n_posts)
        times[:,i+1] = returned[1].reshape(1,n_posts)
        np.savetxt(sub + '_post_data_' + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.csv', data, delimiter=",", fmt='%s', header='Score')
        np.savetxt(sub + '_post_times_' + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.csv', times, delimiter=",", fmt='%s', header='Time')
        print("Scoring cycle completed at",dt.datetime.utcfromtimestamp(time.time()+3600*10.5).strftime('%Y-%m-%d %H:%M:%S'),".")

    for i in range(n_posts):
        try: 
            posted = reddit.submission(id=idlist[i])
            times[i,0] = posted.created_utc
            data[i,0] = 1
        except: 
            times[i,0] = times[i,1] - 3600

    for i in range(n_posts-1,-1,-1):
        if np.sum(np.isnan(data[i,:])) != 0:
            data = np.delete(data,i,0)
            times = np.delete(times,i,0)      

    np.savetxt(sub + '_post_data_clean_' + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.csv', data, delimiter=",", fmt='%s', header='Score')
    np.savetxt(sub + '_post_times_clean_' + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.csv', times, delimiter=",", fmt='%s', header='Time')

In [12]:
comment_data('all',20,1)

20 comments found at 2019-01-18 14:00:25 attempting to track for 1 hours.
Scoring cycle completed at 2019-01-18 14:04:24 .
